# 导包

In [1]:
import torch
import os,re
import pickle
from collections import defaultdict
from transformers import AutoTokenizer
import transformers
from tqdm import tqdm
import torch
import torch.nn as nn
import math
from transformers import AutoModel
tokenizer=AutoTokenizer.from_pretrained("/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/",do_lower_case=False)
import json
from copy import deepcopy
import random
import time
import pandas as pd
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample
from collections import Counter
from copy import deepcopy
from nlp_basictasks.modules.transformers import BertTokenizer
device=torch.device('cuda')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 加载数据

In [2]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/private_test.json'))

原始数据有3000个样本


In [3]:
data[0]

{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'topic_entity': '刘质平'}

# 读取知识图谱

In [9]:
kg_folder='../knowledge/small_knowledge/'

ent2id = {}
with open(os.path.join(kg_folder, 'entities.dict')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    ent2id[l[0].strip()] = len(ent2id)

rel2id = {}
with open(os.path.join(kg_folder, 'relations.dict')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    rel2id[l[0].strip()] = int(l[1])

triples = []
bad_count=0
with open(os.path.join(kg_folder, 'Knowledge.txt')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    try:
        s = ent2id[l[0].strip()]
        p = rel2id[l[1].strip()]
        o = ent2id[l[2].strip()]
        triples.append((s, p, o))
    except:
        bad_count+=1
triples = torch.LongTensor(triples)

Tsize = len(triples)
Esize = len(ent2id)
num_relations = len(rel2id)
print(Tsize,Esize,num_relations)
idx = torch.LongTensor([i for i in range(Tsize)])
Msubj = torch.sparse.FloatTensor(
    torch.stack((idx, triples[:,0])), torch.FloatTensor([1] * Tsize), torch.Size([Tsize, Esize]))
Mobj = torch.sparse.FloatTensor(
    torch.stack((idx, triples[:,2])), torch.FloatTensor([1] * Tsize), torch.Size([Tsize, Esize]))
Mrel = torch.sparse.FloatTensor(
    torch.stack((idx, triples[:,1])), torch.FloatTensor([1] * Tsize), torch.Size([Tsize, num_relations]))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 189599/189599 [00:00<00:00, 983865.49it/s]


189599 146724 4524


# utils

In [20]:
def convert_tokens_to_ids(topic_entity,question,answer=None):
    question=question.replace('<'+topic_entity+'>','NE')
    #print(question)
    head=[ent2id[topic_entity]]
    token_ids=tokenizer(question.strip(), max_length=64, padding='max_length', return_tensors="pt")
    if answer==None:
        return head,token_ids
    else:
        ans_ids=[ent2id[answer]]
        return head,token_ids,ans_ids
    
def toOneHot(indices):
    indices = torch.LongTensor(indices)
    vec_len = len(ent2id)
    one_hot = torch.FloatTensor(vec_len)
    one_hot.zero_()
    one_hot.scatter_(0, indices, 1)
    return one_hot

# 加载模型

In [16]:
bert_name='/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/'
class TransferNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.num_steps = 1#2

        self.bert_encoder = AutoModel.from_pretrained(bert_name, return_dict=True)
        dim_hidden = self.bert_encoder.config.hidden_size

        self.step_encoders = []
        for i in range(self.num_steps):
            m = nn.Sequential(
                nn.Linear(dim_hidden, dim_hidden),
                nn.Tanh()
            )
            self.step_encoders.append(m)
            self.add_module('step_encoders_{}'.format(i), m)

        self.rel_classifier = nn.Linear(dim_hidden, num_relations)

        self.hop_selector = nn.Linear(dim_hidden, self.num_steps)
        self.loss_fn=nn.MSELoss()

    def follow(self, e, r):
        x = torch.sparse.mm(Msubj, e.t()) * torch.sparse.mm(Mrel, r.t())
        return torch.sparse.mm(Mobj.t(), x).t() # [bsz, Esize]

    def forward(self, heads, questions, answers=None, entity_range=None):
        q = self.bert_encoder(**questions)
        q_embeddings, q_word_h = q.pooler_output, q.last_hidden_state # (bsz, dim_h), (bsz, len, dim_h)

        device = heads.device
        last_e = heads
        word_attns = []
        rel_probs = []
        ent_probs = []
        for t in range(self.num_steps):
            cq_t = self.step_encoders[t](q_embeddings) # [bsz, dim_h]
            q_logits = torch.sum(cq_t.unsqueeze(1) * q_word_h, dim=2) # [bsz, max_q]
            q_dist = torch.softmax(q_logits, 1) # [bsz, max_q]
            q_dist = q_dist * questions['attention_mask'].float()
            q_dist = q_dist / (torch.sum(q_dist, dim=1, keepdim=True) + 1e-6) # [bsz, max_q]
            word_attns.append(q_dist)
            ctx_h = (q_dist.unsqueeze(1) @ q_word_h).squeeze(1) # [bsz, dim_h]

            rel_logit = self.rel_classifier(ctx_h) # [bsz, num_relations]
            # rel_dist = torch.softmax(rel_logit, 1) # bad
            rel_dist = torch.sigmoid(rel_logit)
            rel_probs.append(rel_dist)

            # sub, rel, obj = self.triples[:,0], self.triples[:,1], self.triples[:,2]
            # sub_p = last_e[:, sub] # [bsz, #tri]
            # rel_p = rel_dist[:, rel] # [bsz, #tri]
            # obj_p = sub_p * rel_p
            # last_e = torch.index_add(torch.zeros_like(last_e), 1, obj, obj_p)

            last_e = self.follow(last_e, rel_dist) # faster than index_add

            # reshape >1 scores to 1 in a differentiable way
            m = last_e.gt(1).float()
            z = (m * last_e + (1-m)).detach()
            last_e = last_e / z

            ent_probs.append(last_e)

        if self.num_steps>1:
            hop_res = torch.stack(ent_probs, dim=1) # [bsz, num_hop, num_ent]
            hop_attn = torch.softmax(self.hop_selector(q_embeddings), dim=1).unsqueeze(2) # [bsz, num_hop, 1]
            last_e = torch.sum(hop_res * hop_attn, dim=1) # [bsz, num_ent]

        if not self.training:
            return {
                'e_score': last_e,
                'word_attns': word_attns,
                'rel_probs': rel_probs,
                'ent_probs': ent_probs
                #'hop_attn': hop_attn.squeeze(2)
            }
        else:
            weight = answers * 99 + 1
            loss = torch.sum(entity_range * weight * torch.pow(last_e - answers, 2)) / torch.sum(entity_range * weight)
            #loss = self.loss_fn(last_e,answers)
            return {'loss': loss}


In [17]:
model = TransferNet()
model.load_state_dict(torch.load('/home/xhsun/Desktop/KG/kgCLUE/save_dir/model.pt',map_location='cpu'))

Some weights of the model checkpoint at /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/ were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

# 测试模型

In [21]:
model=model.eval()
bad_count=0
for i in tqdm(range(len(data))):
    example=data[i]
    question,topic_entity=example['question'],example['topic_entity']
    if topic_entity not in ent2id:
        data[i].update({"e_score":None,'rel_score':None})
        bad_count+=1
    else:
        head,token_ids=convert_tokens_to_ids(topic_entity,question,answer=None)
        #print(head,token_ids,ans_ids)
        one_hot_head=toOneHot(head)
        with torch.no_grad():
            result=model(*(one_hot_head.unsqueeze(0),token_ids))
        e_score=result['e_score']#(num_entities)
        rel_score=result['rel_probs'][0]#(num_relations)
        data[i].update({"e_score":e_score,'rel_score':rel_score})
    
#    scores,idx=torch.max(e_score,dim=1)
#     score=scores.tolist()[0]
#     predict_id=idx.tolist()[0]
#     predict_answer=id2ent[predict_id]
#     all_predict_examples.append({'score':score,'idx':idx,'predict_answer':predict_answer})
        
#     if predict_answer==answer:
#         correct+=1
print(bad_count)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [01:45<00:00, 28.52it/s]

16


In [24]:
id2ent={k:v for v,k in ent2id.items()}
id2rel={k:v for v,k in rel2id.items()}
for i in tqdm(range(len(data))):
    example=data[i]
    question,topic_entity=example['question'],example['topic_entity']
    if topic_entity not in ent2id:
        data[i].update({"predict_ent":topic_entity,"ent_score":0.0,"predict_rel":"UNK","rel_score":0.0})
    else:
        e_score=example['e_score']
        entity_scores,idx=torch.max(e_score,dim=1)
        ent_score=entity_scores.tolist()[0]
        predict_id=idx.tolist()[0]
        predict_ent=id2ent[predict_id]
        
        r_score=example['rel_score']
        rel_scores,idx=torch.max(r_score,dim=1)
        rel_score=rel_scores.tolist()[0]
        predict_id=idx.tolist()[0]
        predict_rel=id2rel[predict_id]        
        
        data[i].update({"predict_ent":predict_ent,'ent_score':float(ent_score),
                       "predict_rel":predict_rel,'rel_score':float(rel_score)})

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 14951.05it/s]


# 保存提交

In [35]:
submit_examples=[]
for example in data:
    id_,question,topic_entity,predict_ent,predict_rel=example['id'],example['question'],example['topic_entity'],example['predict_ent'],example['predict_rel']
    if topic_entity not in ent2id:
        print(example)
        predict_ent=topic_entity
        predict_rel='UNK'
    ans=str(topic_entity)+" ||| "+str(predict_rel)+" ||| "+str(predict_ent)
    submit_examples.append({"id":id_,"answer":ans})

{'id': 374, 'question': '谁知道进取性优生学叫什么？', 'topic_entity': '进取性优生', 'e_score': None, 'rel_score': 0.0, 'predict_ent': 'UNK', 'ent_score': 0.0, 'predict_rel': 'UNK'}
{'id': 838, 'question': '1945-1949年苏-35战斗机生产了多少台？', 'topic_entity': '1945-1949年苏-35战斗机', 'e_score': None, 'rel_score': 0.0, 'predict_ent': 'UNK', 'ent_score': 0.0, 'predict_rel': 'UNK'}
{'id': 839, 'question': '1945-1949年玉神工艺品有限公司生产了多少台？', 'topic_entity': '1945-1949年玉神工艺品有限公司', 'e_score': None, 'rel_score': 0.0, 'predict_ent': 'UNK', 'ent_score': 0.0, 'predict_rel': 'UNK'}
{'id': 930, 'question': '你知道奈鲁布恩坎的壳需要多少等级吗？', 'topic_entity': '奈鲁布恩坎', 'e_score': None, 'rel_score': 0.0, 'predict_ent': 'UNK', 'ent_score': 0.0, 'predict_rel': 'UNK'}
{'id': 1394, 'question': '企业国有产权无偿划转工作指引出自哪里？', 'topic_entity': '企业国有产权无偿划转工作', 'e_score': None, 'rel_score': 0.0, 'predict_ent': 'UNK', 'ent_score': 0.0, 'predict_rel': 'UNK'}
{'id': 1803, 'question': '0-3岁好习惯儿歌有什么ISBN有谁知道吗', 'topic_entity': '0-3岁好习惯', 'e_score': None, 'rel_score': 0.0, 'pre

In [36]:
submit_examples

[{'id': 0, 'answer': '刘质平 ||| 师承 ||| 李叔同'},
 {'id': 1, 'answer': '南关町 ||| 混缩 ||| なんかんまち'},
 {'id': 2, 'answer': '新道寺女子高中 ||| 日文名称 ||| 高中'},
 {'id': 3, 'answer': '海部市 ||| 日文名称 ||| Ama-shi'},
 {'id': 4, 'answer': '八代市 ||| 日文名称 ||| 熊本县'},
 {'id': 5, 'answer': '三泽市 ||| 日文名称 ||| Misawa-shi'},
 {'id': 6, 'answer': '芥川龙之介奖 ||| 日文名称 ||| 日本'},
 {'id': 7, 'answer': '尤莉 ||| 日文 ||| 三堂镇'},
 {'id': 8, 'answer': '临海女子高中 ||| 日文 ||| 高中'},
 {'id': 9, 'answer': '子实体 ||| 制作材料 ||| 高等真菌'},
 {'id': 10, 'answer': '涨紧轮 ||| 混缩 ||| 主要由固定壳体、张紧臂组成'},
 {'id': 11, 'answer': '卡板 ||| 主要 ||| 双面型。单面型。平板型：间隙型'},
 {'id': 12, 'answer': '苑里车站 ||| 建筑结构 ||| 岛式月台、岸式月台各一座'},
 {'id': 13, 'answer': '急流槽 ||| 混缩 ||| 进口、槽身和出口'},
 {'id': 14, 'answer': '煌斑岩 ||| 制作材料 ||| 煌斑结构、全自形粒状结构'},
 {'id': 15, 'answer': '涅墨西斯 ||| 建筑结构 ||| 三堂镇'},
 {'id': 16, 'answer': '顶棚 ||| 混缩 ||| 面层、基层、吊杆'},
 {'id': 17, 'answer': '单体壳 ||| 主要材料 ||| 迈凯轮'},
 {'id': 18, 'answer': '仙台城 ||| 建筑结构 ||| 伊达政宗'},
 {'id': 19, 'answer': '东风6型内燃机车 ||| 混缩 ||| 大连机车车辆厂、通用电气'},
 {

In [37]:
with open("../kgclue_predict.json","w") as f:
    for predict_example in submit_examples:
        f.write(json.dumps(predict_example,ensure_ascii=False)+'\n')